In [ ]:
!unzip '/content/drive/MyDrive/DeepFashion/Category and Attribute Prediction Benchmark/Img/img.zip'

Streaming output truncated to the last 5000 lines.
  inflating: img/Woven_Colorblock_Romper/img_00000034.jpg  
  inflating: img/Woven_Colorblock_Romper/img_00000035.jpg  
  inflating: img/Woven_Colorblock_Romper/img_00000036.jpg  
  inflating: img/Woven_Colorblock_Romper/img_00000037.jpg  
  inflating: img/Woven_Colorblock_Romper/img_00000038.jpg  
  inflating: img/Woven_Crossback_Dress/img_00000001.jpg  
  inflating: img/Woven_Crossback_Dress/img_00000002.jpg  
  inflating: img/Woven_Crossback_Dress/img_00000003.jpg  
  inflating: img/Woven_Crossback_Dress/img_00000004.jpg  
  inflating: img/Woven_Crossback_Dress/img_00000005.jpg  
  inflating: img/Woven_Crossback_Dress/img_00000006.jpg  
  inflating: img/Woven_Crossback_Dress/img_00000007.jpg  
  inflating: img/Woven_Crossback_Dress/img_00000008.jpg  
  inflating: img/Woven_Crossback_Dress/img_00000009.jpg  
  inflating: img/Woven_Crossback_Dress/img_00000010.jpg  
  inflating: img/Woven_Crossback_Dress/img_00000011.jpg  
  inflating

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import gc 
import pandas as pd
import os
import gdown
import zipfile
import time
from google.colab import drive
%matplotlib inline

In [ ]:
import shutil
import os
import re
import cv2

# will use them for creating custom directory iterator
import numpy as np
from six.moves import range

# regular expression for splitting by whitespace
splitter = re.compile("\s+")
base_path = '/content/img'

In [ ]:
def process_folders():
    # Read the relevant annotation file and preprocess it
    # Assumed that the annotation files are under '<project folder>/data/anno' path
    with open('/content/drive/MyDrive/datasets/dataset/Eval/list_eval_partition.txt', 'r') as eval_partition_file:
        list_eval_partition = [line.rstrip('\n') for line in eval_partition_file][2:]
        list_eval_partition = [splitter.split(line) for line in list_eval_partition]
        list_all = [(v[0][4:], v[0].split('/')[1].split('_')[-1], v[1]) for v in list_eval_partition]

    # Put each image into the relevant folder in train/test/validation folder
    for element in list_all:
        if not os.path.exists(os.path.join(base_path, element[2])):
            print("creating path")
            os.mkdir(os.path.join(base_path, element[2]))
        if not os.path.exists(os.path.join(os.path.join(base_path, element[2]), element[1])):
            os.mkdir(os.path.join(os.path.join(base_path, element[2]), element[1]))
        if not os.path.exists(os.path.join(os.path.join(os.path.join(os.path.join(base_path, element[2]), element[1])),
                              element[0].split('/')[0])):
            os.mkdir(os.path.join(os.path.join(os.path.join(os.path.join(base_path, element[2]), element[1])),
                     element[0].split('/')[0]))
        shutil.move(os.path.join(base_path, element[0]),
                    os.path.join(os.path.join(os.path.join(base_path, element[2]), element[1]), element[0]))

In [ ]:
process_folders()

creating path
creating path
creating path


In [ ]:
def create_dict_bboxes(list_all, split='train'):
    lst = [(line[0], line[1], line[3], line[2]) for line in list_all if line[2] == split]
    lst = [("".join(line[0].split('/')[0] + '/' + line[3] + '/' + line[1] + line[0][3:]), line[1], line[2]) for line in lst]
    lst_shape = [cv2.imread('./' + line[0]).shape for line in lst]
    lst = [(line[0], line[1], (round(line[2][0] / shape[0], 2), round(line[2][1] / shape[1], 2), round(line[2][2] / shape[0], 2), round(line[2][3] / shape[1], 2))) for line, shape in zip(lst, lst_shape)]
    dict_ = {"/".join(line[0].split('/')[2:]): {'origin': {'x': line[2][0], 'y': line[2][1]}, 'width': line[2][2], 'height': line[2][3]} for line in lst}
    return dict_

In [ ]:
def get_dict_bboxes():
    with open('/content/drive/MyDrive/datasets/dataset/Anno/list_category_img.txt', 'r') as category_img_file, \
            open('/content/drive/MyDrive/datasets/dataset/Eval/list_eval_partition.txt', 'r') as eval_partition_file, \
            open('/content/drive/MyDrive/datasets/dataset/Anno/list_bbox.txt', 'r') as bbox_file:
        list_category_img = [line.rstrip('\n') for line in category_img_file][2:]
        list_eval_partition = [line.rstrip('\n') for line in eval_partition_file][2:]
        list_bbox = [line.rstrip('\n') for line in bbox_file][2:]

        list_category_img = [splitter.split(line) for line in list_category_img]
        list_eval_partition = [splitter.split(line) for line in list_eval_partition]
        list_bbox = [splitter.split(line) for line in list_bbox]

        list_all = [(k[0], k[0].split('/')[1].split('_')[-1], v[1], (int(b[1]), int(b[2]), int(b[3]), int(b[4])))
                    for k, v, b in zip(list_category_img, list_eval_partition, list_bbox)]

        list_all.sort(key=lambda x: x[1])

        dict_train = create_dict_bboxes(list_all)
        dict_val = create_dict_bboxes(list_all, split='val')
        dict_test = create_dict_bboxes(list_all, split='test')

        return dict_train, dict_val, dict_test

In [ ]:
import keras
from keras.models import Model
from keras.layers import Dense
from keras.regularizers import l2
from keras.optimizers import SGD
from keras.applications.resnet50 import ResNet50
from keras.preprocessing import image
from keras.preprocessing.image import DirectoryIterator, ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, EarlyStopping, TensorBoard
from keras import backend as K

In [ ]:
model_resnet = ResNet50(weights='imagenet', include_top=False, pooling='avg')

94773248/94765736 [==============================] - 1s 0us/step


In [ ]:
model_resnet.summary()

In [ ]:
for layer in model_resnet.layers[:-12]:
    # 6 - 12 - 18 have been tried. 12 is the best.
    layer.trainable = False

In [ ]:
x = model_resnet.output
x = Dense(512, activation='elu', kernel_regularizer=l2(0.001))(x) # THIS IS ELU, NOT RELU
y = Dense(46, activation='softmax', name='img')(x)

In [ ]:
x_bbox = model_resnet.output
x_bbox = Dense(512, activation='relu', kernel_regularizer=l2(0.001))(x_bbox)
x_bbox = Dense(128, activation='relu', kernel_regularizer=l2(0.001))(x_bbox)
bbox = Dense(4, kernel_initializer='normal', name='bbox')(x_bbox)

In [ ]:
final_model = Model(inputs=model_resnet.input,
                    outputs=[y, bbox])

In [ ]:
final_model.summary()

In [ ]:
opt = SGD(lr=0.0001, momentum=0.9, nesterov=True)

In [ ]:
final_model.compile(optimizer=opt,
                    loss={'img': 'categorical_crossentropy',
                          'bbox': 'mean_squared_error'},
                    metrics={'img': ['accuracy', 'top_k_categorical_accuracy'], # default: top-5
                             'bbox': ['mse']})

In [ ]:
train_datagen = ImageDataGenerator(rotation_range=30.,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   horizontal_flip=True)
test_datagen = ImageDataGenerator()


In [ ]:
class DirectoryIteratorWithBoundingBoxes(DirectoryIterator):
    def __init__(self, directory, image_data_generator, bounding_boxes: dict = None, target_size=(256, 256),
                 color_mode: str = 'rgb', classes=None, class_mode: str = 'categorical', batch_size: int = 32,
                 shuffle: bool = True, seed=None, data_format=None, save_to_dir=None,
                 save_prefix: str = '', save_format: str = 'jpeg', follow_links: bool = False):
        super().__init__(directory, image_data_generator, target_size, color_mode, classes, class_mode, batch_size,
                         shuffle, seed, data_format, save_to_dir, save_prefix, save_format, follow_links)
        self.bounding_boxes = bounding_boxes

    def next(self):
        """
        # Returns
            The next batch.
        """
        with self.lock:
            index_array = next(self.index_generator)
        # The transformation of images is not under thread lock
        # so it can be done in parallel
        batch_x = np.zeros((len(index_array),) + self.image_shape, dtype=K.floatx())
        locations = np.zeros((len(batch_x),) + (4,), dtype=K.floatx())

        grayscale = self.color_mode == 'grayscale'
        # build batch of image data
        for i, j in enumerate(index_array):
            fname = self.filenames[j]
            img = image.load_img(os.path.join(self.directory, fname),
                                 grayscale=grayscale,
                                 target_size=self.target_size)
            x = image.img_to_array(img, data_format=self.data_format)
            x = self.image_data_generator.random_transform(x)
            x = self.image_data_generator.standardize(x)
            batch_x[i] = x

            if self.bounding_boxes is not None:
                bounding_box = self.bounding_boxes[fname]
                locations[i] = np.asarray(
                    [bounding_box['origin']['x'], bounding_box['origin']['y'], bounding_box['width'],
                     bounding_box['height']],
                    dtype=K.floatx())
        # optionally save augmented images to disk for debugging purposes
        # build batch of labels
        if self.class_mode == 'sparse':
            batch_y = self.classes[index_array]
        elif self.class_mode == 'binary':
            batch_y = self.classes[index_array].astype(K.floatx())
        elif self.class_mode == 'categorical':
            batch_y = np.zeros((len(batch_x), 46), dtype=K.floatx())
            for i, label in enumerate(self.classes[index_array]):
                batch_y[i, label] = 1.
        else:
            return batch_x

        if self.bounding_boxes is not None:
            return batch_x, [batch_y, locations]
        else:
            return batch_x, batch_y

In [ ]:
dict_train, dict_val, dict_test = get_dict_bboxes()

train_iterator = DirectoryIteratorWithBoundingBoxes("./img/train", train_datagen, bounding_boxes=dict_train, target_size=(200, 200))

test_iterator = DirectoryIteratorWithBoundingBoxes("./img/val", test_datagen, bounding_boxes=dict_val,target_size=(200, 200))

Found 209222 images belonging to 46 classes.
Found 40000 images belonging to 46 classes.


In [ ]:
model1 = keras.models.load_model('/content/drive/MyDrive/models/model.h5')

In [ ]:
model1.summary()

In [ ]:
lr_reducer = ReduceLROnPlateau(monitor='val_loss',
                               patience=12,
                               factor=0.5,
                               verbose=1)

In [ ]:
tensorboard = TensorBoard(log_dir='./logs')

In [ ]:
early_stopper = EarlyStopping(monitor='val_loss',
                              patience=30,
                              verbose=1)

In [ ]:
checkpoint = ModelCheckpoint('/content/drive/MyDrive/models/model.h5')

In [ ]:
def custom_generator(iterator):
    while True:
        batch_x, batch_y = iterator.next()
        yield (batch_x, batch_y)

In [ ]:
final_model.fit(custom_generator(train_iterator),
                steps_per_epoch=2000,
                epochs=200, validation_data=custom_generator(test_iterator),
                validation_steps=200,
                verbose=2,
                shuffle=True,
                callbacks=[lr_reducer, checkpoint, early_stopper, tensorboard],
                )

Epoch 1/200
2000/2000 - 705s - loss: 4.2308 - img_loss: 2.3320 - bbox_loss: 0.0636 - img_accuracy: 0.3636 - img_top_k_categorical_accuracy: 0.6933 - bbox_mse: 0.0636 - val_loss: 3.7725 - val_img_loss: 1.8923 - val_bbox_loss: 0.0521 - val_img_accuracy: 0.4620 - val_img_top_k_categorical_accuracy: 0.8005 - val_bbox_mse: 0.0521
Epoch 2/200
2000/2000 - 674s - loss: 3.8292 - img_loss: 1.9631 - bbox_loss: 0.0453 - img_accuracy: 0.4416 - img_top_k_categorical_accuracy: 0.7886 - bbox_mse: 0.0453 - val_loss: 3.5961 - val_img_loss: 1.7386 - val_bbox_loss: 0.0439 - val_img_accuracy: 0.5036 - val_img_top_k_categorical_accuracy: 0.8313 - val_bbox_mse: 0.0439
Epoch 3/200
2000/2000 - 682s - loss: 3.6984 - img_loss: 1.8514 - bbox_loss: 0.0406 - img_accuracy: 0.4714 - img_top_k_categorical_accuracy: 0.8110 - bbox_mse: 0.0406 - val_loss: 3.5254 - val_img_loss: 1.6859 - val_bbox_loss: 0.0403 - val_img_accuracy: 0.5150 - val_img_top_k_categorical_accuracy: 0.8452 - val_bbox_mse: 0.0403
Epoch 4/200
2000/20

In [ ]:
model1.fit(custom_generator(train_iterator),
                steps_per_epoch=2000,
                epochs=200, validation_data=custom_generator(test_iterator),
                validation_steps=200,
                verbose=2,
                shuffle=True,
                callbacks=[lr_reducer, checkpoint, early_stopper, tensorboard],
                )

Epoch 1/200
2000/2000 - 708s - loss: 3.0698 - img_loss: 1.4744 - bbox_loss: 0.0288 - img_img_accuracy: 0.5648 - img_img_top_k_categorical_accuracy: 0.8784 - bbox_bbox_mse: 0.0288 - val_loss: 2.9944 - val_img_loss: 1.4080 - val_bbox_loss: 0.0260 - val_img_img_accuracy: 0.5913 - val_img_img_top_k_categorical_accuracy: 0.8892 - val_bbox_bbox_mse: 0.0260
Epoch 2/200
2000/2000 - 673s - loss: 3.0507 - img_loss: 1.4676 - bbox_loss: 0.0288 - img_img_accuracy: 0.5662 - img_img_top_k_categorical_accuracy: 0.8807 - bbox_bbox_mse: 0.0288 - val_loss: 2.9641 - val_img_loss: 1.3901 - val_bbox_loss: 0.0258 - val_img_img_accuracy: 0.5983 - val_img_img_top_k_categorical_accuracy: 0.8911 - val_bbox_bbox_mse: 0.0258
Epoch 3/200
2000/2000 - 670s - loss: 3.0378 - img_loss: 1.4677 - bbox_loss: 0.0280 - img_img_accuracy: 0.5663 - img_img_top_k_categorical_accuracy: 0.8786 - bbox_bbox_mse: 0.0280 - val_loss: 2.9658 - val_img_loss: 1.4033 - val_bbox_loss: 0.0265 - val_img_img_accuracy: 0.5906 - val_img_img_top_

In [ ]:
model2 = keras.models.load_model('/content/drive/MyDrive/models/model.h5')

In [ ]:
model2.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None, None,  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, None, None, 3 0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, None, None, 6 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, None, None, 6 256         conv1_conv[0][0]                 
______________________________________________________________________________________________

In [ ]:
lr_reducer = ReduceLROnPlateau(monitor='val_loss',
                               patience=12,
                               factor=0.5,
                               verbose=1)

In [ ]:
tensorboard = TensorBoard(log_dir='./logs')

In [ ]:
early_stopper = EarlyStopping(monitor='val_loss',
                              patience=30,
                              verbose=1)

In [ ]:
checkpoint = ModelCheckpoint('/content/drive/MyDrive/models/model.h5')

In [ ]:
model2.fit(custom_generator(train_iterator),
                steps_per_epoch=2000,
                epochs=200, validation_data=custom_generator(test_iterator),
                validation_steps=200,
                verbose=2,
                shuffle=True,
                callbacks=[lr_reducer, checkpoint, early_stopper, tensorboard],
                )

Epoch 1/200
2000/2000 - 712s - loss: 2.7711 - img_loss: 1.3744 - bbox_loss: 0.0265 - img_img_img_accuracy: 0.5925 - img_img_img_top_k_categorical_accuracy: 0.8943 - bbox_bbox_bbox_mse: 0.0265 - val_loss: 2.6878 - val_img_loss: 1.2998 - val_bbox_loss: 0.0232 - val_img_img_img_accuracy: 0.6194 - val_img_img_img_top_k_categorical_accuracy: 0.9034 - val_bbox_bbox_bbox_mse: 0.0232
Epoch 2/200
2000/2000 - 677s - loss: 2.7650 - img_loss: 1.3793 - bbox_loss: 0.0263 - img_img_img_accuracy: 0.5884 - img_img_img_top_k_categorical_accuracy: 0.8933 - bbox_bbox_bbox_mse: 0.0263 - val_loss: 2.6963 - val_img_loss: 1.3187 - val_bbox_loss: 0.0236 - val_img_img_img_accuracy: 0.6170 - val_img_img_img_top_k_categorical_accuracy: 0.9025 - val_bbox_bbox_bbox_mse: 0.0236
Epoch 3/200
2000/2000 - 673s - loss: 2.7436 - img_loss: 1.3686 - bbox_loss: 0.0262 - img_img_img_accuracy: 0.5923 - img_img_img_top_k_categorical_accuracy: 0.8958 - bbox_bbox_bbox_mse: 0.0262 - val_loss: 2.7031 - val_img_loss: 1.3348 - val_bb

In [ ]:
model2 = keras.models.load_model('/content/drive/MyDrive/models/model.h5')

In [ ]:
lr_reducer = ReduceLROnPlateau(monitor='val_loss',
                               patience=12,
                               factor=0.5,
                               verbose=1)

In [ ]:
tensorboard = TensorBoard(log_dir='./logs')

In [ ]:
early_stopper = EarlyStopping(monitor='val_loss',
                              patience=30,
                              verbose=1)

In [ ]:
checkpoint = ModelCheckpoint('/content/drive/MyDrive/models/model.h5')

In [ ]:
model2.fit(custom_generator(train_iterator),
                steps_per_epoch=2000,
                epochs=50, validation_data=custom_generator(test_iterator),
                validation_steps=200,
                verbose=2,
                shuffle=True,
                callbacks=[lr_reducer, checkpoint, early_stopper, tensorboard],
                )

Epoch 1/50
2000/2000 - 648s - loss: 2.3309 - img_loss: 1.2733 - bbox_loss: 0.0242 - img_img_img_img_img_accuracy: 0.6194 - img_img_img_img_img_top_k_categorical_accuracy: 0.9085 - bbox_bbox_bbox_bbox_bbox_mse: 0.0242 - val_loss: 2.3149 - val_img_loss: 1.2633 - val_bbox_loss: 0.0222 - val_img_img_img_img_img_accuracy: 0.6342 - val_img_img_img_img_img_top_k_categorical_accuracy: 0.9058 - val_bbox_bbox_bbox_bbox_bbox_mse: 0.0222
Epoch 2/50
2000/2000 - 616s - loss: 2.3108 - img_loss: 1.2607 - bbox_loss: 0.0248 - img_img_img_img_img_accuracy: 0.6214 - img_img_img_img_img_top_k_categorical_accuracy: 0.9123 - bbox_bbox_bbox_bbox_bbox_mse: 0.0248 - val_loss: 2.2950 - val_img_loss: 1.2511 - val_bbox_loss: 0.0225 - val_img_img_img_img_img_accuracy: 0.6394 - val_img_img_img_img_img_top_k_categorical_accuracy: 0.9097 - val_bbox_bbox_bbox_bbox_bbox_mse: 0.0225
Epoch 3/50
2000/2000 - 614s - loss: 2.3151 - img_loss: 1.2733 - bbox_loss: 0.0243 - img_img_img_img_img_accuracy: 0.6177 - img_img_img_img_i

KeyboardInterrupt: ignored

In [ ]:
main_model = keras.models.load_model('/content/drive/MyDrive/models/model.h5')
main_model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None, None,  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, None, None, 3 0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, None, None, 6 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, None, None, 6 256         conv1_conv[0][0]                 
______________________________________________________________________________________________

In [ ]:
from PIL import Image

In [ ]:
out = main_model.predict(im)

In [ ]:
embedding = Model(inputs=main_model.input,
                    outputs=main_model.layers[-6].output)
embedding.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None, None,  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, None, None, 3 0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, None, None, 6 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, None, None, 6 256         conv1_conv[0][0]                 
______________________________________________________________________________________________

In [ ]:
embed = embedding(im)
embed

<tf.Tensor: shape=(1, 2048), dtype=float32, numpy=
array([[0.4315781 , 0.11283883, 0.49542013, ..., 0.675453  , 0.11951996,
        0.11737636]], dtype=float32)>

In [ ]:
embed.numpy()[0]

array([0.4315781 , 0.11283883, 0.49542013, ..., 0.675453  , 0.11951996,
       0.11737636], dtype=float32)

In [ ]:
!unzip /content/drive/MyDrive/PRODUCT.zip

Archive:  /content/drive/MyDrive/PRODUCT.zip
   creating: PRODUCT/
  inflating: __MACOSX/._PRODUCT      
  inflating: PRODUCT/WMSH000688_1.JPG  
  inflating: __MACOSX/PRODUCT/._WMSH000688_1.JPG  
  inflating: PRODUCT/WMTS000868_5.jpg  
  inflating: __MACOSX/PRODUCT/._WMTS000868_5.jpg  
  inflating: PRODUCT/WMTS000909_1.jpg  
  inflating: __MACOSX/PRODUCT/._WMTS000909_1.jpg  
  inflating: PRODUCT/WMSS000787_1.JPG  
  inflating: __MACOSX/PRODUCT/._WMSS000787_1.JPG  
  inflating: PRODUCT/WMTS000765_6.jpg  
  inflating: __MACOSX/PRODUCT/._WMTS000765_6.jpg  
  inflating: PRODUCT/WMSH002088_3.JPG  
  inflating: __MACOSX/PRODUCT/._WMSH002088_3.JPG  
  inflating: PRODUCT/WMSH000695_4.JPG  
  inflating: __MACOSX/PRODUCT/._WMSH000695_4.JPG  
  inflating: PRODUCT/WMTS000816_3.jpg  
  inflating: __MACOSX/PRODUCT/._WMTS000816_3.jpg  
  inflating: PRODUCT/WMTS000798_6.jpg  
  inflating: __MACOSX/PRODUCT/._WMTS000798_6.jpg  
  inflating: PRODUCT/WMTS000853_3.jpg  
  inflating: __MACOSX/PRODUCT/._WMTS

In [ ]:
from PIL import Image

img = Image.open('/content/PRODUCT/WMTS000886_3.jpg')
im = np.asarray(img)
im = im.reshape(1,im.shape[0],im.shape[1],im.shape[2])

In [ ]:
print(os.listdir('/content/PRODUCT')[0][:-4])

WMSH000605_4


In [ ]:
path = '/content/PRODUCT/'
embed = []
for i in os.listdir(path):
  embed.append(embedding(np.expand_dims(np.asarray(Image.open(path+i)),axis=0)).numpy()[0])

In [ ]:
print(embed[0].shape)

(2048,)


In [ ]:
exp = embed

In [ ]:
exp = np.vstack(exp)

In [ ]:
exp.shape

(1176, 2048)

In [ ]:
print(exp)

[[0.5241164  0.8396236  0.08855595 ... 0.15680674 0.05632399 0.3725018 ]
 [0.71580255 0.6547816  0.2224994  ... 0.07707518 0.29191798 0.17878546]
 [0.67402613 0.50191194 0.06684426 ... 0.12607396 0.3893357  0.12239122]
 ...
 [0.72947097 0.7262496  0.05760834 ... 0.29595855 0.36483163 0.1705557 ]
 [0.97027236 0.8015573  0.1637579  ... 0.17641489 0.21533297 0.20742407]
 [0.60125375 0.43636212 0.2803494  ... 3.022519   0.19618872 0.23363721]]


In [ ]:
dir = os.listdir(path)

In [ ]:
import pickle

try:
	id = open('image_id', 'wb')
	pickle.dump(dir, id)
	id.close()

except:
	print("Something went wrong")


In [ ]:
embed[0]

array([0.5241164 , 0.8396236 , 0.08855595, ..., 0.15680674, 0.05632399,
       0.3725018 ], dtype=float32)

In [ ]:
embedd = {}
n = np.arange(0,1176)
addr = os.listdir(path)
for i in n:
  embedd[addr[i][:-4]] = exp[i]

In [ ]:
import pickle

try:
	emb = open('embeddings', 'wb')
	pickle.dump(embedd, emb)
	emb.close()

except:
	print("Something went wrong")


In [ ]:
print(embedd['WMSH0002040_1'])

[0.5349963  0.53996795 0.02717499 ... 0.19937603 0.04783098 0.24760167]


In [ ]:
loaded = open('/content/embeddings','rb')
embed1 = pickle.load(loaded)
embed1

{'WMSH000605_4': array([0.5241164 , 0.8396236 , 0.08855595, ..., 0.15680674, 0.05632399,
        0.3725018 ], dtype=float32),
 'WMTS000795_4': array([0.71580255, 0.6547816 , 0.2224994 , ..., 0.07707518, 0.29191798,
        0.17878546], dtype=float32),
 'WMSH002102_3': array([0.67402613, 0.50191194, 0.06684426, ..., 0.12607396, 0.3893357 ,
        0.12239122], dtype=float32),
 'WMTS000935_4': array([0.8015882 , 0.64785725, 0.25249493, ..., 0.61013603, 0.23391312,
        0.12654692], dtype=float32),
 'WMTS000854_5': array([0.9682146 , 0.45963526, 0.22866307, ..., 0.31836537, 0.14736764,
        0.14421557], dtype=float32),
 'WMSH000703_1': array([0.5511203 , 0.6880429 , 0.10170221, ..., 0.12609531, 0.12841862,
        0.19646022], dtype=float32),
 'WMSH000697_5': array([0.66518277, 0.5839838 , 0.08994867, ..., 0.5140883 , 0.16147818,
        0.11008158], dtype=float32),
 'WMTS000890_3': array([0.91651756, 0.47723186, 0.17796575, ..., 0.30518338, 0.07190724,
        0.21288124], dtype=fl

In [ ]:
print(embed1['WMSH0002040_6'])

[0.62377733 0.63021255 0.04973249 ... 0.06324961 0.22569521 0.13676687]
